In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

%matplotlib inline


In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
data = pd.read_sql("select * from fire_and_air_data where fire_year > 2012", con=engine)
data.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,date,overall_aqi,site_name,year,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,distance
0,2013-01-01,66,Redwood City,2013,86781.0,VOYAGER,2013-01-01,2013-01-01,0.1,A,2013.0,304.658747
1,2013-01-01,66,Redwood City,2013,89616.0,None,2013-01-01,2013-01-01,1.0,B,2013.0,395.689071
2,2013-01-01,66,Redwood City,2013,89889.0,None,2013-01-01,2013-01-01,1.0,B,2013.0,439.916187
3,2013-01-02,61,Livermore,2013,87714.0,WALTERS,2013-01-02,2013-01-02,0.3,B,2013.0,524.945435
4,2013-01-05,85,San Rafael,2013,88404.0,BECKWITH,2013-01-05,2013-01-05,1.0,B,2013.0,67.354106


In [4]:
year = pd.read_sql("select distinct fire_year from fire_and_air_data", con=engine)
year.head()

,fire_year
0,2001.0
1,NaN
2,2002.0
3,2003.0
4,2004.0


In [5]:
data = data[['date', 'overall_aqi', 'fire_id', 'size', 'distance']]

In [6]:
bins = [0, 50, 100, 150, 200, 250, 10000000000000000]

In [7]:
labels = ['within 50', 'within 100', 'within 150', 'within 200', 'within 250', 'bigger then 250']

In [8]:
data['bins'] = pd.cut(data['distance'], bins=bins, labels=labels)

In [9]:
data.head()

,date,overall_aqi,fire_id,size,distance,bins
0,2013-01-01,66,86781.0,0.1,304.658747,bigger then 250
1,2013-01-01,66,89616.0,1.0,395.689071,bigger then 250
2,2013-01-01,66,89889.0,1.0,439.916187,bigger then 250
3,2013-01-02,61,87714.0,0.3,524.945435,bigger then 250
4,2013-01-05,85,88404.0,1.0,67.354106,within 100


In [37]:
# Return a grouped by object (with no aggregate operation)...
# BY Bins, then date
new_group = data.groupby(['date','bins'])

In [51]:
data.columns.values.tolist()

['date', 'overall_aqi', 'fire_id', 'size', 'distance', 'bins']

In [53]:
engine.execute('''

SELECT *
FROM
    (SELECT date, SUM(distance) total_distance
    FROM acres_and_air_data
    GROUP BY 1
    ORDER BY 1 ASC) acres_fire_size
JOIN
    (SELECT DISTINCT date, distance, overall_aqi
    FROM acres_and_air_data) unique_aqi
ON acres_fire_size.date = unique_aqi.date
LIMIT 10
''').fetchall()

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'distance' in 'field list'") [SQL: '\n\nSELECT *\nFROM\n    (SELECT date, SUM(distance) total_distance\n    FROM acres_and_air_data\n    GROUP BY 1\n    ORDER BY 1 ASC) acres_fire_size\nJOIN\n    (SELECT DISTINCT date, distance, overall_aqi\n    FROM acres_and_air_data) unique_aqi\nON acres_fire_size.date = unique_aqi.date\nLIMIT 10\n'] (Background on this error at: http://sqlalche.me/e/2j85)

In [39]:
#For the grouped, we then sum. 
new_group_agg = new_group.agg(np.average)
new_group_agg.head(30)

overall_aqi   fire_id  size     distance
date       bins                                                     
2013-01-01 within 50                NaN       NaN   NaN          NaN
           within 100               NaN       NaN   NaN          NaN
           within 150               NaN       NaN   NaN          NaN
           within 200               NaN       NaN   NaN          NaN
           within 250               NaN       NaN   NaN          NaN
           bigger then 250        198.0  266286.0   2.1  1140.264005
2013-01-02 within 50                NaN       NaN   NaN          NaN
           within 100               NaN       NaN   NaN          NaN
           within 150               NaN       NaN   NaN          NaN
           within 200               NaN       NaN   NaN          NaN
           within 250               NaN       NaN   NaN          NaN
           bigger then 250         61.0   87714.0   0.3   524.945435
2013-01-05 within 50                NaN       NaN   NaN          NaN
           within 100              85.0   88404.0   1.0    67.354106
           within 150              85.0   89500.0   1.0   112.686684
           within 200              85.0   88581.0   0.1   176.053789
           within 250               NaN       NaN   NaN          NaN
           bigger then 250        425.0  447762.0   5.1  2019.195742
2013-01-06 within 50                NaN       NaN   NaN          NaN
           within 100              55.0   88405.0   1.0    67.358127
           within 150               NaN       NaN   NaN          NaN
           within 200               NaN       NaN   NaN          NaN
           within 250               NaN       NaN   NaN          NaN
           bigger then 250         55.0   89614.0   1.0   386.359957
2013-01-07 within 50                NaN       NaN   NaN          NaN
           within 100               NaN       NaN   NaN          NaN
           within 150               NaN       NaN   NaN          NaN
           within 200               NaN       NaN   NaN          NaN
           within 250               NaN       NaN   NaN          NaN
           bigger then 250         58.0   87105.0   0.1   311.870950

In [28]:
new_group = bin_group.sum().dropna()

In [29]:
new_group.sum().head()

overall_aqi    7.118750e+05
fire_id        1.236969e+09
size           9.855783e+05
distance       2.757074e+06
dtype: float64

In [36]:
new_group.head(20)

,date,overall_aqi,fire_id,size,distance,bins
0,2013-01-01,66,86781.0,0.10,304.658747,bigger then 250
1,2013-01-01,66,89616.0,1.00,395.689071,bigger then 250
2,2013-01-01,66,89889.0,1.00,439.916187,bigger then 250
3,2013-01-02,61,87714.0,0.30,524.945435,bigger then 250
4,2013-01-05,85,88404.0,1.00,67.354106,within 100
5,2013-01-05,85,88581.0,0.10,176.053789,within 200
6,2013-01-05,85,89215.0,1.00,409.294978,bigger then 250
7,2013-01-05,85,89315.0,2.00,409.435828,bigger then 250
8,2013-01-05,85,89500.0,1.00,112.686684,within 150
9,2013-01-05,85,89508.0,1.00,405.850962,bigger then 250


In [ ]:
# data.plot(x="distance", y="overall_aqi", kind="scatter")

In [ ]:
 data.plot.bar(x='bins', y='overall_aqi', rot=0)

In [ ]:
new_group.head()

In [ ]:
# new_group.reset_index().plot.bar(x='bins', y='overall_aqi', rot=0)

In [ ]:
combined_group = data.groupby(['date','fire_id'])

In [ ]:
combined_group.head()

In [ ]:
# verify that there's never more then one record for a particular fire-id on a particular day
combined_group.size().max()

In [ ]:
grouped_by_date = data.groupby(['date'])

In [ ]:
grouped_by_date.describe().head()

In [ ]:
grouped_by_date.sum()

In [ ]:
acres_per_day_df = data.groupby('date')['size'].sum().to_frame().reset_index()

In [ ]:
acres_per_day_df.head()

In [ ]:
# Manually spot-check a date

data[data['date'] == '2014-01-03']

In [ ]:
air_data_df = pd.read_sql("select * from aqi_data", con=engine)
air_data_df.head()

In [ ]:
merged_data_df = pd.merge(air_data_df, acres_per_day_df, on='date')

In [ ]:
merged_data_df.head()

In [ ]:
merged_data_df.to_sql(name="acres_and_air_data", con=engine, if_exists="replace")